<a href="https://colab.research.google.com/github/Edgar-La/hacking-civico/blob/master/proyecto/proyecto-hacking-civico.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# __Titulo:__
En este documento encontras lo siguiente:

* Descripcion:

* Selección y obtención de los datos

* Exploración de los datos

* Visualización de los datos

* Conclusión 

* Referencias


## __Descripcion.__
Información de los registros de la deuda pública contraída vigente (acreedor, acreditado, monto, garantías, tasas, comisiones, plazos, amortizaciones, destino, etc.).
Auditoría Superior del Estado de Guanajuato

## __Seleccion y obtencion de datos.__

In [50]:
#Importar paquetes de Python
import pandas as pd
import numpy as np
import altair as alt
import statistics

In [ ]:
#Para descargar los datos
!wget http://www.aseg.gob.mx/descarga/deuda_publica.csv

In [70]:
#Convertir datos en DataFrame

# Especificamos el nombre del archivo por abrir
nombre_archivo = 'deuda_publica.csv'
# Utilizamos la función de Pandas para cargar un archivo CSV
Datos = pd.read_csv(nombre_archivo, encoding = 'latin-1')

In [71]:
# Limpieza del nombre de las columnas, remover espacios y '.'
Datos.columns = Datos.columns.str.replace(' ', '_')
Datos.columns = Datos.columns.str.replace('.', '')

#Limpieza de lo que contienen ciertas columnas, como '$', ',', y también pasamos los NaN -> 0 con fillna(0) indicando la columna
for t in Datos.Monto_original_contratado_en_pesos:
    Datos.Monto_original_contratado_en_pesos = Datos.Monto_original_contratado_en_pesos.str.replace('$', '')
    Datos.Monto_original_contratado_en_pesos = Datos.Monto_original_contratado_en_pesos.str.replace(',', '') 
Datos['Monto_original_contratado_en_pesos'] = Datos['Monto_original_contratado_en_pesos'].fillna(0)                         ### Nuevo comando  ###

for t in Datos.Monto_dispuesto_en_pesos:
    Datos.Monto_dispuesto_en_pesos = Datos.Monto_dispuesto_en_pesos.str.replace('$', '')
    Datos.Monto_dispuesto_en_pesos = Datos.Monto_dispuesto_en_pesos.str.replace(',', '')
    #Datos.Monto_dispuesto_en_pesos = Datos.Monto_dispuesto_en_pesos.str.replace('NaN', '0')
    Datos.Monto_dispuesto_en_pesos = Datos.Monto_dispuesto_en_pesos.str.replace('Total', '0')
Datos['Monto_dispuesto_en_pesos'] = Datos['Monto_dispuesto_en_pesos'].fillna(0)

for t in Datos.Saldo_en_pesos	:
    Datos.Saldo_en_pesos = Datos.Saldo_en_pesos.str.replace('$', '')
    Datos.Saldo_en_pesos = Datos.Saldo_en_pesos.str.replace(',', '')
Datos['Saldo_en_pesos'] = Datos['Saldo_en_pesos'].fillna(0)

Datos.head()   #Muestra cabecera

,Ente_Acreditado,No_de_Crédito,No_de_inscripción_SHCP,Registro_Estatal,Ambito,Institución_Acreedora,Tipo_de_Obligación,Clase_del_titulo,Monto_original_contratado_en_pesos,Monto_dispuesto_en_pesos,Saldo_en_pesos,Tasa,Fecha_de_Contratación,Plazo_máximo_(meses),Fecha_de_Vencimiento,Garantía,Destino
0,Gobierno del Estado de Guanajuato,NaN,167/2002,124/02,Estatal,"Banco del Bajío, S.A.",Contrato de Apertura de Crédito Simple,Pesos,233000000.00,232851337.75,15597340.75,TIIE +1.00,09/12/2002,180,27/11/2017,Participaciones Federales,Se destinará precisa y exclusivamente para fin...
1,Gobierno del Estado de Guanajuato,1466.0,169/02,125/02,Estatal,Banobras S.N.C,Contrato de apertura de crédito simple y const...,Pesos,289037000.00,289037000.00,52301933.78,FOAEM Nominal + 1.50,18/12/2002,240,10/03/2023,Participaciones Federales,Se destinará precisa y exclusivamente para fin...
2,Gobierno del Estado de Guanajuato,1467.0,169/02,125/02,Estatal,Banobras S.N.C,Contrato de apertura de crédito simple y const...,UDIS,289037000.00,289037000.00,84325399.36,FOAEM Real + 1.50,18/12/2002,240,10/03/2023,Participaciones Federales,Se destinará precisa y exclusivamente para fin...
3,Tarimoro,4470.0,276/2004,140/04,Municipal,Banobras S.N.C,Contrato de apertura de crédito simple y const...,Pesos,3132842.97,3132842.97,671324.13,TIIE * 1.120,11/11/2004,180,23/12/2019,Participaciones Federales,Se destinará única y exclusivamente en los sig...
4,Gobierno del Estado de Guanajuato,5768.0,234/2005,147/05,Estatal,Banobras S.N.C,Contrato de Apertura de Crédito Simple,Pesos,1184414400.00,1184414400.00,97109245.00,CETES 182 + 1.34,20/05/2005,216,01/02/2022,Participaciones Federales,Obra pública


In [72]:
#Podemos notar que los valores que nos interesan siguen siendo tipo object
Datos.dtypes

Ente_Acreditado                        object
No_de_Crédito                         float64
No_de_inscripción_SHCP                 object
Registro_Estatal                       object
Ambito                                 object
Institución_Acreedora                  object
Tipo_de_Obligación                     object
Clase_del_titulo                       object
Monto_original_contratado_en_pesos     object
Monto_dispuesto_en_pesos               object
Saldo_en_pesos                         object
Tasa                                   object
Fecha_de_Contratación                  object
Plazo_máximo_(meses)                   object
Fecha_de_Vencimiento                   object
Garantía                               object
Destino                                object
dtype: object

In [73]:
#Convertimos las columnas que nos interesan a numeric con pd.to_numeric()
Datos['Monto_original_contratado_en_pesos'] = pd.to_numeric(Datos['Monto_original_contratado_en_pesos'])
Datos['Monto_dispuesto_en_pesos'] = pd.to_numeric(Datos['Monto_dispuesto_en_pesos'])
Datos['Saldo_en_pesos'] = pd.to_numeric(Datos['Saldo_en_pesos'])

Datos.dtypes

Ente_Acreditado                        object
No_de_Crédito                         float64
No_de_inscripción_SHCP                 object
Registro_Estatal                       object
Ambito                                 object
Institución_Acreedora                  object
Tipo_de_Obligación                     object
Clase_del_titulo                       object
Monto_original_contratado_en_pesos    float64
Monto_dispuesto_en_pesos              float64
Saldo_en_pesos                        float64
Tasa                                   object
Fecha_de_Contratación                  object
Plazo_máximo_(meses)                   object
Fecha_de_Vencimiento                   object
Garantía                               object
Destino                                object
dtype: object

## __Exploracion de los datos.__

In [74]:
#Datos.groupby('Ente_Acreditado').sum()
Datos.groupby('Institución_Acreedora').Ente_Acreditado.count()      #Filtrado por institucion acredora
#Datos.query('Ente_Acreditado == "Guanajuato"').head()

Institución_Acreedora
BBVA Bancomer                                             6
Banamex                                                   7
Banco del Bajío, S.A.                                    19
Banobras S.N.C                                           19
Banorte                                                   1
Concesionaria del Acueducto El Zapotillo S.A. de C.V.     1
HSBC                                                      2
Scotiabank                                                1
Name: Ente_Acreditado, dtype: int64

In [80]:
#Filtro para Guanajuato
Datos.query('Ente_Acreditado == "Gobierno del Estado de Guanajuato"').Monto_original_contratado_en_pesos.value_counts()

1.184414e+09    5
5.000000e+08    5
2.890370e+08    2
1.700000e+09    1
3.367197e+08    1
2.000000e+09    1
2.359502e+08    1
6.289892e+08    1
2.152000e+09    1
2.330000e+08    1
6.440000e+08    1
1.600000e+09    1
Name: Monto_original_contratado_en_pesos, dtype: int64

In [81]:
Datos_L = Datos.groupby('Ente_Acreditado').Monto_original_contratado_en_pesos.describe();
#Datos_L = Datos_L.convert_dtypes;
Datos_L

,count,mean,std,min,25%,50%,75%,max
Ente_Acreditado,,,,,,,,
Abasolo,1.0,1.300000e+07,NaN,1.300000e+07,1.300000e+07,1.300000e+07,1.300000e+07,1.300000e+07
Apaseo el Grande,1.0,1.980342e+07,NaN,1.980342e+07,1.980342e+07,1.980342e+07,1.980342e+07,1.980342e+07
Celaya,1.0,3.500000e+08,NaN,3.500000e+08,3.500000e+08,3.500000e+08,3.500000e+08,3.500000e+08
"Comisión Estatal del Agua de Guanajuato\nSistema de Agua Potable y Alcantarillado de León, Gto.",1.0,3.310000e+09,NaN,3.310000e+09,3.310000e+09,3.310000e+09,3.310000e+09,3.310000e+09
"Comité Municipal de Agua Potable y Alcantarillado de Salamanca, Gto.",1.0,4.300000e+07,NaN,4.300000e+07,4.300000e+07,4.300000e+07,4.300000e+07,4.300000e+07
Comonfort,1.0,1.986088e+07,NaN,1.986088e+07,1.986088e+07,1.986088e+07,1.986088e+07,1.986088e+07
Cuerámaro,2.0,7.750000e+06,6.363961e+05,7.300000e+06,7.525000e+06,7.750000e+06,7.975000e+06,8.200000e+06
Doctor Mora,1.0,7.493849e+06,NaN,7.493849e+06,7.493849e+06,7.493849e+06,7.493849e+06,7.493849e+06
Gobierno del Estado de Guanajuato,21.0,8.824193e+08,5.997558e+08,2.330000e+08,5.000000e+08,6.289892e+08,1.184414e+09,2.152000e+09


In [82]:
Datos_M = Datos.groupby('Ente_Acreditado').Monto_original_contratado_en_pesos.value_counts()
#Datos_M = pd.DataFrame(data = Datos_M);
Datos_M

Ente_Acreditado                                                                                  Monto_original_contratado_en_pesos
Abasolo                                                                                          1.300000e+07                          1
Apaseo el Grande                                                                                 1.980342e+07                          1
Celaya                                                                                           3.500000e+08                          1
Comisión Estatal del Agua de Guanajuato\nSistema de Agua Potable y Alcantarillado de León, Gto.  3.310000e+09                          1
Comité Municipal de Agua Potable y Alcantarillado de Salamanca, Gto.                             4.300000e+07                          1
Comonfort                                                                                        1.986088e+07                          1
Cuerámaro                                     

In [ ]:
#Datos_K = Datos.query('Ente_Acreditado == "Guanajuato"').Monto_dispuesto_en_pesos.value_counts()
#Datos_K

## __Visualizacion de los datos.__

In [83]:
Grafico_1 = alt.Chart(Datos).mark_bar().encode(
    x = alt.X('Fecha_de_Vencimiento:N'),
    y = alt.Y('Ente_Acreditado:N'),
    color = 'Ente_Acreditado',
)
Grafico_1

alt.Chart(...)

In [84]:
Grafico_2 = alt.Chart(Datos).mark_bar().encode(
    x = 'Ente_Acreditado',
    y = 'Monto_original_contratado_en_pesos',
    color = 'Ente_Acreditado'
)
Grafico_2

alt.Chart(...)

In [85]:
alt.Chart(Datos).mark_bar().encode(
    x = alt.X('Institución_Acreedora'),
    y = alt.X('count(Institución_Acreedora)'),
    color = 'Institución_Acreedora',
)

alt.Chart(...)

## __Conclusion.__

## __Referencias.__